# DSCI Group Project -- Group 42

## Research Question

In [3]:
library(tidyverse)
library(dplyr)
library(tidymodels)
set.seed(6666)

players <- read_csv("players.csv")


# clean_players <- players |>
#     mutate(subscribe = as.factor(subscribe),
#            experience = as.factor(experience),
#            gender = as.factor(gender)) |>
#     select(-hashedEmail, -Age, -name)

clean_players <- players |>
    mutate(subscribe = as.factor(subscribe)) |>
    mutate(experience = as.numeric(factor(experience, 
                        levels = c("Beginner", "Amateur", "Pro", "Veteran")))) |>
    mutate(Age = as.numeric(Age)) |>
  
    select(-hashedEmail, -gender, -name)

clean_players

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.6     ✔ rsample      1.2.1
✔ dials        1.3.0     ✔ tune         1.1.2
✔ infer        1.0.7     ✔ workflows    1.1.4
✔ modeldata    1.4.0     ✔ workflowsets 1.0.1
✔ parsnip      1.2.1     ✔ yardstick    1.3.1
✔ recipes      1.1.0     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

experience,subscribe,played_hours,Age
<dbl>,<fct>,<dbl>,<dbl>
3,TRUE,30.3,9
4,TRUE,3.8,17
4,FALSE,0.0,17
2,TRUE,0.7,21
NA,TRUE,0.1,21
2,TRUE,0.0,17
NA,TRUE,0.0,19
2,FALSE,0.0,21
2,TRUE,0.1,47


## Setting Up Training and Testing Data

In [4]:
players_split <- initial_split(clean_players, prop = 0.7, strata = subscribe)
players_train <- training(players_split)
players_test <- testing(players_split)

## Creating a Recipe and Model

In [5]:
players_recipe <- recipe(subscribe ~ ., data = players_train) |>
    step_scale(all_predictors()) |>
    step_center(all_predictors())
players_recipe

knn_spec_tuned <- nearest_neighbor(weight_func = "rectangular", neighbors = tune()) |>
    set_engine("kknn") |>
    set_mode("classification")



── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 3



── Operations 

• Scaling for: all_predictors()

• Centering for: all_predictors()



## Using 5 Fold Cross Validation and Train the Model

In [ ]:
players_vfold <- vfold_cv(players_train, v = 5, strata = subscribe)
k_vals <- tibble(neighbors = seq(from = 1, to = 103, by = 1))

knn_tuned_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_spec_tuned) |>
    tune_grid(resamples = players_vfold, grid = k_vals) |>
    collect_metrics()

→ A | error:   only 0's may be mixed with negative subscripts

There were issues with some computations   A: x1



## Visualize Accuracy with Different K Values

In [ ]:
accuracy <- knn_tuned_fit |>
    filter(.metric == "accuracy")
accuracy_plot <- accuracy |>
    ggplot(aes(x = neighbors, y = mean)) +
    geom_point() +
    labs(x = "Number of Neighbors",
         y = "Mean Value of Accuracy") +
    ggtitle("The Relationship between Number of Neighbors (k) and Their Mean Value") +
    theme(text = element_text(size = 20))
accuracy_plot

## Find the Best K and Fitting it in Training Data

In [13]:
best_k <- accuracy |>
    filter(mean == max(mean)) |>
    pull(neighbors)
knn_spec_best <- nearest_neighbor(weight_func = "rectangular", neighbor = 29) |>
    set_engine("kknn") |>
    set_mode("classification")
knn_best_fit <- workflow() |>
    add_recipe(players_recipe) |>
    add_model(knn_spec_best) |>
    fit(players_test)


## Predict Using Testing Data

In [14]:
subscribe_predictions <- predict(knn_best_fit, players_test) |>
    bind_cols(players_test)
subscribe_metrics <- subscribe_predictions |>
    metrics(truth = subscribe, estimate = .pred_class)
subscribe_metrics
subscribe_predictions

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.7333333
kap,binary,0.0000000


.pred_class,experience,subscribe,played_hours,gender
<fct>,<fct>,<fct>,<dbl>,<fct>
TRUE,Veteran,FALSE,0.0,Male
TRUE,Amateur,FALSE,0.0,Male
TRUE,Pro,TRUE,0.0,Male
TRUE,Amateur,TRUE,0.2,Male
TRUE,Veteran,TRUE,0.0,Non-binary
TRUE,Regular,TRUE,0.3,Male
TRUE,Amateur,FALSE,0.1,Female
TRUE,Beginner,TRUE,0.0,Male
TRUE,Amateur,TRUE,1.8,Male
